In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
%autoreload

In [3]:
import os
os.chdir('/home/jaentrouble/adipose_train3')
print(os.getcwd())

/home/jaentrouble/adipose_train3


In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import imageio as io
import json

img_names = os.listdir('data/done')
img = []
img_name_dict = {}
for idx, name in enumerate(img_names):
    img.append(io.imread('data/done/'+name))
    img_name_dict[name] = idx

json_names = os.listdir('data/save')
data = []
for name in json_names:
    with open('data/save/'+name,'r') as j:
        data.extend(json.load(j))
for datum in data :
    datum['image'] = img_name_dict[datum['image']]

In [6]:
!git pull
%autoreload

Already up to date.


In [7]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [8]:
from model_trainer import run_training
from encoder_models import *
from box_models import *
from model_lr import *
from multiprocessing import Process
from functools import partial

In [9]:
encoder_f = hr_5_3_0_gap
img_size = (400,320)
box_f = partial(dense_128_4_normoutpos, image_size=img_size)
lr_f = lr_mul_inv
name = 'hr_5_3_0_d128_normoutpos_test'
epochs = 10
batch_size = 10
steps_per_epoch = len(data)//batch_size
mixed_float = True

In [10]:
kwargs = dict(
    encoder_f = encoder_f,
    box_f = box_f,
    lr_f = lr_f,
    name = name,
    epochs = epochs,
    steps_per_epoch=steps_per_epoch,
    mixed_float=mixed_float,
    batch_size=batch_size,
    img=img,
    data=data,
    img_size=img_size,
    notebook= True,
)

In [11]:
run_training(**kwargs)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5



Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.


ValueError: in user code:

    /home/jaentrouble/adipose_train3/env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/jaentrouble/adipose_train3/model_trainer.py:66 call  *
        return self.box_model([encoded, inputs['pos']], training=training)
    /home/jaentrouble/adipose_train3/env/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:985 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    /home/jaentrouble/adipose_train3/env/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:386 call
        inputs, training=training, mask=mask)
    /home/jaentrouble/adipose_train3/env/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:508 _run_internal_graph
        outputs = node.layer(*args, **kwargs)
    /home/jaentrouble/adipose_train3/env/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:985 _